In [6]:
import pickle
path = r'./IEMOCAP_emotion_cause_features.pkl'

In [20]:
loaded = pickle.load(open(path, 'rb'), encoding='latin1')

videoIDs, videoSpeakers, videoLabels, causeLabels, causeLabels2, causeLabels3, videoText,\
videoAudio, videoVisual, videoSentence, trainVid,\
testVid = loaded

'''
label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5} 
# disgust가 없음, fear가 없고 frustration이 있음
'''

"\nlabel index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5} \n# disgust가 없음, fear가 없고 frustration이 있음\n"

In [8]:
def get_dataset_dictionary(vids):
    label_mapping = {0: 'happy', 1: 'sad', 2: 'neutral', 3: 'angry', 4: 'excited', 5: 'frustrated'}
    # label_mapping = {0: 'happy', 1: 'sad', 2: 'neutral', 3: 'angry', 4: 'fear', 5: 'disgust'}
    # # 사실은 fear로 표시된 건 excited고, disgust로 표시되는 건 frustrated인데, 기존 프리프로세싱이랑 겹치니까 안 겹치도록 저렇게 마킹함
    full_dataset = {}
    key_list  = list(vids)
    for key in key_list:
        a_dialog = []
        turn_num = 1
        dialog_full_utterances = [_ for _ in videoSentence[key]]
        for speaker, sentence, emotion, cl1, cl2, cl3 in zip(videoSpeakers[key], videoSentence[key], videoLabels[key], causeLabels[key], causeLabels2[key], causeLabels3[key]):
            a_utterance = {}
            
            a_utterance['turn'] = turn_num
            a_utterance['speaker'] = "A" if speaker=="M" else "B"
            a_utterance['utterance'] = sentence
            a_utterance['emotion'] = label_mapping[emotion]
            
            cl_list = [cl1, cl2, cl3]
            cl_nonzero = [cl for cl in cl_list if cl != 0]
            
            if a_utterance['emotion'] != 'neutral':
                a_utterance['expanded emotion cause span'] = [dialog_full_utterances[_-1] for _ in cl_nonzero]
                cl_nonzero = ['b' if value>turn_num else value for value in cl_nonzero]
                a_utterance['expanded emotion cause evidence'] = cl_nonzero
                a_utterance['type'] = [
                    'no_context' for _ in cl_nonzero
                ]
            turn_num += 1
            a_dialog.append(a_utterance)
            
        full_dataset[key] = [a_dialog]

    return full_dataset

In [9]:
Vids = trainVid

full_dataset = get_dataset_dictionary(Vids)
full_dataset

{'Ses04F_impro02': [[{'turn': 1,
    'speaker': 'A',
    'utterance': "So you're leaving tomorrow.",
    'emotion': 'sad',
    'expanded emotion cause span': ["So you're leaving tomorrow.",
     "I mean we both knew that this was going to happen.  It's just that I feel that, you know, I don't want Susie growing up without a mother."],
    'expanded emotion cause evidence': [1, 'b'],
    'type': ['no_context', 'no_context']},
   {'turn': 2,
    'speaker': 'B',
    'utterance': 'Yeah.  They just called.',
    'emotion': 'sad',
    'expanded emotion cause span': ["I don't know what to say.  I don't want to go but I don't have a lot of choice."],
    'expanded emotion cause evidence': ['b'],
    'type': ['no_context']},
   {'turn': 3,
    'speaker': 'A',
    'utterance': 'This is really-this is unfair.',
    'emotion': 'sad',
    'expanded emotion cause span': ["So you're leaving tomorrow."],
    'expanded emotion cause evidence': [1],
    'type': ['no_context']},
   {'turn': 4,
    'speak

In [50]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
        
Vids = trainVid
full_dataset_train = get_dataset_dictionary(Vids)
json.dump(full_dataset_train, open('ConvECPE_train_originalsplit.json', 'w'), indent="\t", cls=NpEncoder)


Vids = testVid
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_test_originalsplit.json', 'w'), indent="\t", cls=NpEncoder)

In [24]:
(len(full_dataset_test) + len(full_dataset_train))*0.8

120.80000000000001

In [46]:
import random
all_vids = set.union(trainVid, testVid)
train_sets = random.sample(all_vids, k=121)

/tmp/ipykernel_1282896/2981197724.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_sets = random.sample(all_vids, k=121)


In [47]:
b = set.difference(all_vids, train_sets)
test_sets = random.sample(b, k=15)
valid_sets = set.difference(b, test_sets)

/tmp/ipykernel_1282896/1646907373.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_sets = random.sample(b, k=15)


In [48]:
(len(train_sets), len(test_sets), len(valid_sets))

(121, 15, 15)

In [49]:
Vids = train_sets
full_dataset_train = get_dataset_dictionary(Vids)
json.dump(full_dataset_train, open('ConvECPE_train.json', 'w'), indent="\t", cls=NpEncoder)


Vids = test_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_test.json', 'w'), indent="\t", cls=NpEncoder)

Vids = valid_sets
full_dataset_test = get_dataset_dictionary(Vids)
json.dump(full_dataset_test, open('ConvECPE_valid.json', 'w'), indent="\t", cls=NpEncoder)